In [2]:
from __future__ import annotations
import numpy as np
from scipy.signal import convolve2d
import tkinter as tk
from itertools import product
from math import floor, prod
import random

In [41]:
from random import choices


class Game:
    def __init__(
            self,
            grid: Grid,
        ) -> None:
        self.grid = grid
        # What the player can see
        self.player_grid_view = np.zeros(grid.grid_shape(), dtype=bool)
    
    def action(self, x: int, y: int):# Discover new part (or not new) of the grid
        if np.sum(self.player_grid_view) == 0:
            # First action of the player, we move the grid such that the player input is on a empty cell
            self.grid.move_to_empty(x, y)

        self.player_grid_view = np.logical_or(
            self.player_grid_view,
            self.grid.discover(x, y),
        )
    
    def is_ended(self):
        result = self.result()
        return result is not None
    
    def result(self):
        # Discovered a mine
        if np.any(np.logical_and(self.player_grid_view, self.grid.mines)):
            return False
        
        # Discovered has many tile than free tiles
        return None if self.grid.n_bomb + np.sum(self.player_grid_view) < prod(self.grid.grid_shape()) else True
    
    def visible_grid(self):
        return self.grid.grid*(self.player_grid_view), self.player_grid_view

        


class Grid:
    def __init__(self, n_row: int, n_col: int, bomb_percent:float) -> None:
        self.grid = np.zeros((n_row, n_col))

        self.mines = np.zeros((n_row, n_col), dtype=bool)
        # Place bombs
        self.n_bomb = floor(n_row*n_col*bomb_percent)
        bomb_coordinates = random.sample(list(product(range(n_row), range(n_col))), self.n_bomb)
        for x, y in bomb_coordinates:
            self.mines[x, y] = True

        # Compute the adjacents bombs
        self.grid = convolve2d(self.mines, np.ones((3, 3)), mode='same')

    def move_to_empty(self, x: int, y: int):
        # Find a good spot to move
        values = np.copy(self.grid)
        values[self.mines] = np.max(self.grid)+1
        min_value = np.min(values)
        possible_destinations = np.argwhere(values == min_value)
        destination: np.ndarray = choices(possible_destinations, k=1)[0]

        # Slide the grid
        self.slide_grid(*(np.array([x, y]) - destination))

    def slide_grid(self, delta_x: int, delta_y: int):
        self.mines = np.roll(
            np.roll(
                self.mines,
                delta_x,
                axis=0,
            ),
            delta_y,
            axis=1,
        )

        self.update()


    def update(self):
        # Update the number of adjacents if the mines array have been changed
        self.grid = convolve2d(self.mines, np.ones((3, 3)), mode='same')

    def grid_shape(self):
        return self.mines.shape
    
    def discover(self, x:int, y: int):
        if self.grid[x, y] > 0: # Cover the case of a mine
            result = np.zeros_like(self.mines)
            result[x, y] = True
            return result
        
        # Expand the area
        return self.expand(x, y)

    def expand(self, x:int, y:int):
        result = np.zeros_like(self.mines)
        result[x, y] = True
        last_result = np.zeros_like(self.mines)
        while np.any(last_result != result): # Check if stabilized
            last_result = result
            
            # Restrict
            result = np.logical_and(
                result,
                self.grid == 0,
            )
            
            # Expand
            result = convolve2d(result, np.ones((3, 3)), mode='same').astype(bool)

            
        return result
        
class Player_Interface:
    def action(self, grid: np.ndarray, grid_view: np.ndarray) -> tuple[int, int]:
        pass

class Random_Player(Player_Interface):
    def action(self, grid: np.ndarray, grid_view: np.ndarray):
        return random.randint(0, grid.shape[0]-1), random.randint(0, grid.shape[1]-1)
    

class Command_Line_UI:
    def start(self, game: Game, player:Player_Interface):
        print('Game start')
        print(f'Grid size : {game.grid.grid_shape()}')
        while not game.is_ended(): # Player has not discovered all cases
            next_action = player.action(*game.visible_grid())
            print(f'Player plays {next_action}')
            
            game.action(*next_action)
        
        result = game.result()
        if result:
            print('Player win')
        else:
            print('Player lose')
        return result

In [40]:
import tkinter as tk
from tkinter import messagebox
import random

class GUI_User_Inputs:
    def __init__(self, master: tk.Tk | None=None):
        self.master = master if master is not None else tk.Tk()
        self.master.title("Minesweeper")
        self.master.geometry("1440x720")

        # Create the structure to place the game grid
        self.create_widgets()

    def start(self, game: Game):
        # Update grid of the minesweeper
        self.initialize_grid(game)

        # Lunch the app
        self.master.mainloop()

        return game.result()        

    def create_widgets(self):
        self.grid_frame = tk.Frame(self.master)
        self.grid_frame.pack(pady=10)

        self.status_bar = tk.Label(self.master, text=f"Total number of mines : Unknow", bd=1, relief=tk.SUNKEN, anchor=tk.W)
        self.status_bar.pack(side=tk.BOTTOM, fill=tk.X)

        # self.initialize_grid()
        # self.generate_mines(10)

    def initialize_grid(self, game: Game):
        # Initialize flag storage
        self.flags = np.zeros_like(game.player_grid_view)

        # Create the buttons/grid
        self.buttons: list[list[tk.Button]] = []
        for row in range(game.grid.grid_shape()[0]):
            row_buttons = []
            for col in range(game.grid.grid_shape()[1]):
                button = tk.Button(self.grid_frame, width=2, height=1, command=lambda r=row, c=col: self.on_button_click(game, r, c))
                button.grid(row=row, column=col)
                button.bind("<Button-3>", lambda e, r=row, c=col: self.on_right_click(game, r, c))
                row_buttons.append(button)
            self.buttons.append(row_buttons)

        # Upadte the label
        self.status_bar.config(text=f"Total number of mines : {game.grid.n_bomb}")

        self.update_grid(game)

    def on_button_click(self, game: Game, row: int, col: int):
        game.action(row, col)

        self.update_grid(game)

    def on_right_click(self, game: Game, row: int, col: int):
        self.flags[row, col] = not self.flags[row, col]
        self.update_grid(game)

    def update_grid(self, game: Game):
        # Place the numbers on the buttons
        for row in range(game.grid.grid_shape()[0]):
            for col in range(game.grid.grid_shape()[1]):
                button = self.buttons[row][col]

                if not game.player_grid_view[row, col]:
                    if self.flags[row, col]:
                        text_button = 'F'
                        color='yellow'
                    else:
                        text_button = ''
                        color='gray'
                elif game.grid.mines[row, col]:
                    text_button = 'M'
                    color='red'
                else:
                    value = int(game.grid.grid[row, col])
                    text_button = str(value) if value != 0 else ''
                    color='lightgrey'

                button.config(text=text_button, bg=color)

gui = GUI_User_Inputs()
grid = Grid(30, 30, 0.1)
game = Game(grid)

game_result = gui.start(game)
game_result

False

In [45]:


from tkinter import Tk


class GUI_Bot_Inputs(GUI_User_Inputs):
    def __init__(self, master: Tk | None = None):
        super().__init__(master)
        self.master.bind("<FocusIn>", self.on_focus_in)

    def on_focus_in(self, event):
        while not self.game.is_ended():
            next_action = self.player.action(*self.game.visible_grid())
            self.on_button_click(self.game, *next_action)
            self.master.update_idletasks()

    def start(self, game: Game, player: Player_Interface):
        self.game = game
        self.player = player
        return super().start(game)
    
gui_bot = GUI_Bot_Inputs()
grid = Grid(30, 30, 0.1)
game = Game(grid)

game_result = gui_bot.start(game, Random_Player())
game_result

False

In [47]:
from numpy import ndarray
grid = Grid(5, 5, 0.1)
game = Game(grid)
game.action(2, 2)

In [50]:
values, known = game.visible_grid()

values - ~known

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  0.,  0.],
       [-1.,  2.,  1.,  1.,  0.],
       [-1., -1., -1.,  1.,  0.]])

In [66]:
class Minesweeper_bot(Player_Interface):
    def action(self, grid: ndarray, grid_view: ndarray) -> tuple[int, int]:
        self.grid = grid
        self.grid_view = grid_view

        # Only make sense on the ~grid_view array
        self.unkown_region = ~grid_view
        self.known_mines = np.zeros_like(grid_view)
        # self.known_no_mines = np.copy(grid_view) # Useless, if we found a known_no_mines, we return it imediatly

        self.to_inspect = list(product(*[range(l) for l in grid.shape]))
        for x, y in self.to_inspect:
            self.inspect(x, y)

    def all_neighbors(self, x: int, y: int):
        # List all neigbhors
        neighbors = [
            (x+dx, y+dy) 
            for dx in (-1, 0, 1) for dy in (-1, 0, 1) 
            if 0 <= x+dx and x+dx < self.known_mines.shape[0]
            and 0 <= y+dy and y+dy < self.known_mines.shape[1]
            and not (dx==0 and dy==0)
        ]
        return neighbors

    def inspect(self, x: int, y: int):
        # List all neigbhors
        neighbors = self.all_neighbors(x, y)

        if self.grid_view[x, y]:
            # The box is known
            value = self.grid[x, y]
            # Case value == 0 isn't interestng because all the neighbors are values
            if value > 0:
                # Values is bigger than 0
                # Three cases :
                    # The number of unknown is equals to the number of bomb -> all bombs
                    # The number of known bombs is equals to the number of bombs -> other boxes are safe
                    # The number of unknwon and the number of knowned bomb are less than the number of bomb -> no decision

                unknown_boxes = [
                    (i, j)
                    for i, j in neighbors
                    if self.unkown_region[i, j] and not self.known_mines[i, j]
                ]
                if len(unknown_boxes) == value:
                    # All bombs
                    for i, j in neighbors:
                        if self.unkown_region[i, j]:
                            self.known_mines[i, j] = True
                            self.unkown_region[i, j] = False # We know this value now
                            # Might give information to neighbors of the mine
                            self.to_inspect += self.all_neighbors(i, j)
                else:
                    # Check if all the mines have been founded
                    founded_mines = [
                        (i, j)
                        for i, j in neighbors
                        if self.known_mines[i, j]
                    ]
                    if len(founded_mines) == len(unknown_boxes):
                        # All remaining boxes are safe
                        for i, j in neighbors:
                            if self.unkown_region[i, j] and not self.mines[i, j]:
                                return i, j
                    else:
                        # No decision can be made
                        pass

    # We didn't deduce

    print("Didn't manage to find a solution")


Didn't manage to find a solution


In [ ]:

# Remodalisation : Chaque case peut prendre 4 etats : [Connue valeur, Inconnue, Deduite bombe, deduite sans bombe]
# On peut ecrire des règles de transitions :
    # Si le nombre de voisinage Inconue == valeur -> Voisinage transitionne en Deduite bombe
    # Si le nombre de voisinage bombe connue == valeur ->  Voisinage transitionne en Deduite sans bombe

Minesweeper_bot().action(values, known)

In [63]:
values - ~known

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  0.,  0.],
       [-1.,  2.,  1.,  1.,  0.],
       [-1., -1., -1.,  1.,  0.]])